
# 🧭 EDA Notebook Template (Python)

Ten szablon pomoże Ci szybko wystartować z eksploracyjną analizą danych (EDA).
Wypełnij sekcje oznaczone komentarzami `# TODO` i uruchamiaj komórki po kolei.

**Wymagania:** Python 3.9+, pakiety: `pandas`, `numpy`, `matplotlib` (opcjonalnie: `pyarrow` lub `openpyxl` do wczytywania plików).

**Tipy:**
- Traktuj ten notebook jako szkic roboczy – dopisuj wnioski w komórkach Markdown.
- Zachowuj porządek: sekcje `Univariate`, `Bivariate`, `Time Series`, `Feature Quality` itd.
- Nie zakładaj, że dane są czyste – **sprawdzaj wszystko**.


In [ ]:

# === Imports ===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ustawienia wyświetlania
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# Wykresy w notebooku
%matplotlib inline


In [ ]:

# === Konfiguracja ===
# TODO: Podaj ścieżkę do danych (CSV/Parquet/Excel)
DATA_PATH = "data.csv"   # np. "data/transactions.csv"

# Opcje wczytywania (zmień wg potrzeb)
READ_KW = dict(
    sep=",",
    decimal=".",
    encoding="utf-8",
    low_memory=False
)

# Jeżeli korzystasz z dat - ustaw kolumny do parsowania
PARSE_DATES = []  # np. ["order_date"]
DATE_FORMAT = None  # np. "%Y-%m-%d"


In [ ]:

# === Wczytanie danych ===
ext = os.path.splitext(DATA_PATH)[1].lower()

if ext in [".csv", ".txt"]:
    df = pd.read_csv(DATA_PATH, **READ_KW, parse_dates=PARSE_DATES)
elif ext in [".parquet"]:
    df = pd.read_parquet(DATA_PATH)
elif ext in [".xlsx", ".xls"]:
    df = pd.read_excel(DATA_PATH, parse_dates=PARSE_DATES, engine=None)
else:
    raise ValueError(f"Nieobsługiwane rozszerzenie pliku: {ext}")

print("Kształt danych:", df.shape)
df.head()


In [ ]:

# === Szybki rzut oka na strukturę ===
display(df.sample(min(len(df), 5), random_state=42))
display(df.describe(include=[np.number]).T)
display(df.describe(include=['object', 'category']).T)

print("\n--- Info ---")
df.info()


In [ ]:

# === Braki danych (missing values) ===
na_counts = df.isna().sum().sort_values(ascending=False)
na_ratio = (na_counts / len(df)).rename("na_ratio")
missing = pd.concat([na_counts.rename("na_count"), na_ratio], axis=1)
missing[missing.na_count > 0].head(30)


In [ ]:

# === Duplikaty i jakość danych ===
dup_rows = df.duplicated().sum()
print(f"Liczba zduplikowanych wierszy: {dup_rows}")

# TODO: Jeśli masz klucz główny, sprawdź unikalność
PRIMARY_KEY = []  # np. ["order_id"]
if PRIMARY_KEY:
    is_unique = ~df.duplicated(subset=PRIMARY_KEY)
    print(f"Klucz {PRIMARY_KEY} unikalny? ->", bool(is_unique.all()))


In [ ]:

# === Przegląd typów i konwersje ===
df.dtypes

# TODO: Przykłady konwersji:
# num_cols = ["price", "qty"]
# for c in num_cols:
#     df[c] = pd.to_numeric(df[c], errors="coerce")

# cat_cols = ["category"]
# for c in cat_cols:
#     df[c] = df[c].astype("category")

# date_cols = ["order_date"]
# for c in date_cols:
#     df[c] = pd.to_datetime(df[c], format=DATE_FORMAT, errors="coerce")


In [ ]:

# === Univariate: zmienne numeryczne ===
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Kolumny numeryczne:", num_cols)

# Histogramy
for col in num_cols:
    plt.figure()
    df[col].plot(kind="hist", bins=30, edgecolor="black")
    plt.title(f"Histogram: {col}")
    plt.xlabel(col)
    plt.ylabel("Liczność")
    plt.show()

# Statystyki rozszerzone
num_stats = df[num_cols].agg(["count","mean","std","min","median","max"]).T
num_stats


In [ ]:

# === Univariate: zmienne kategoryczne ===
cat_cols = df.select_dtypes(include=["object","category","bool"]).columns.tolist()
print("Kolumny kategoryczne:", cat_cols)

TOP_N = 20

summaries = {}
for col in cat_cols:
    vc = df[col].astype("object").value_counts(dropna=False).head(TOP_N)
    summaries[col] = vc

# Podgląd jednej z kolumn
if cat_cols:
    example_col = cat_cols[0]
    display(df[example_col].value_counts(dropna=False).head(TOP_N))

# TODO: narysuj rozkłady dla wybranych kolumn
# for col in cat_cols[:5]:
#     plt.figure()
#     df[col].value_counts().head(TOP_N).plot(kind="bar")
#     plt.title(f"Top {TOP_N}: {col}")
#     plt.xlabel(col)
#     plt.ylabel("Liczność")
#     plt.show()


In [ ]:

# === Bivariate: korelacje liczbowych ===
if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    corr.round(3)
else:
    print("Za mało kolumn numerycznych do korelacji.")


In [ ]:

# === Sanity checks & potencjalny leakage ===
# TODO: Ustal 'target' jeżeli budujesz model i sprawdź zależności
TARGET = None  # np. "churned"
if TARGET and TARGET in df.columns:
    print("Rozkład targetu:")
    display(df[TARGET].value_counts(normalize=True).rename("share").round(3))


In [ ]:

# === Analiza czasowa (opcjonalnie) ===
# TODO: Wybierz kolumnę daty i grupowanie
DATE_COL = None  # np. "order_date"
if DATE_COL and DATE_COL in df.columns:
    s = df.set_index(DATE_COL).sort_index()
    # Przykładowe agregacje po miesiącu
    monthly = s.resample("M").size().rename("count")
    display(monthly.head())

    plt.figure()
    monthly.plot()
    plt.title("Liczność w czasie (miesięcznie)")
    plt.xlabel("Data")
    plt.ylabel("Liczność")
    plt.show()


In [ ]:

# === Outliers (IQR) ===
def iqr_outliers(series, factor=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - factor * iqr
    upper = q3 + factor * iqr
    return (series < lower) | (series > upper), (lower, upper)

outlier_report = {}
for col in num_cols:
    mask, (low, high) = iqr_outliers(df[col].dropna())
    outlier_report[col] = dict(
        lower=low, upper=high, outliers=int(mask.sum())
    )

pd.DataFrame(outlier_report).T.sort_values("outliers", ascending=False).head(20)


In [ ]:

# === Zapis przetworzonych danych (opcjonalnie) ===
# TODO: Dostosuj przetwarzanie do własnych potrzeb
df_clean = df.copy()

# Przykład: drop duplikatów
# df_clean = df_clean.drop_duplicates()

# Przykład: fillna dla liczbowych
# for c in num_cols:
#     df_clean[c] = df_clean[c].fillna(df_clean[c].median())

# Zapis
# df_clean.to_parquet("data_clean.parquet", index=False)
# albo
# df_clean.to_csv("data_clean.csv", index=False)



## ✍️ Wnioski
- [Dodaj tutaj najważniejsze obserwacje]
- [Wypunktuj problemy jakości danych]
- [Zdefiniuj kolejne kroki: inżynieria cech / model / dashboard]
